In [1]:
import sys
from time import time
import numpy as np

sys.path.append('..')

import matplotlib.pyplot as plt
from dataset.models.tf import VGG7, VGG16
from dataset import B, Pipeline, C, V, Config
from dataset.opensets import MNIST

from grid import Option, KV, Grid

from singlerun import SingleRunning
from multiplerun import Research

import tensorflow as tf

In [2]:
shape = (28, 28, 1)
n_classes = 10

In [3]:
placeholders_config = {
    'images': {'shape': shape,
               'type': 'float32',
               'name': 'reshaped_images'
              },
    'labels': {'classes': n_classes,
               'type': 'int32',
               'transform': 'ohe',
               'name': 'targets'
              }
}

feed_dict = {'images': B('images'),
             'labels': B('labels')}

In [4]:
model_config = {
    'session/config': tf.ConfigProto(gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.6, allow_growth=True)),
    'inputs': placeholders_config,
    'input_block/inputs': 'images',
    'batch_norm': {'momentum': 0.1},
    'output': {'ops': ['accuracy']},
    'loss': 'ce',
    'optimizer': 'Adam'
}

In [5]:
ppl_cfg = Config(model_config=model_config)

In [14]:
op1 = Option(KV('model_config/head/units', 'A'), [KV([100, 100, 10], 'small')])
op2 = Option(KV('model_config/body/block', 'B'), 
             [KV(dict(layout='cna'), 'cna'), 
              KV(dict(layout='can'), 'can'), 
              KV(dict(layout='acn'), 'acn')])
op3 = Option(KV('model_config/batch_norm/momentum', 'momentum'), [0.1, 0.5])
op4 = Option('model', [KV(VGG7, 'VGG7')])
grid_config = op2 * op4

In [15]:
mnist = MNIST()
train_preproc = mnist.train.p.run(batch_size=8, shuffle=True, n_epochs=None, lazy=True)
test_preproc = mnist.test.p.run(batch_size=10, shuffle=True, n_epochs=None, lazy=True)
train_ppl = (Pipeline()
             .init_variable('loss', init_on_each_run=list)
             .init_model('dynamic', C('model'), 'model', C('model_config'))
             .train_model('model', feed_dict=feed_dict, fetches='loss', save_to=V('loss'), mode='a')
            )

test_ppl = (Pipeline()
             .init_variable('accuracy', init_on_each_run=list)
             .import_model('model', C('import_model_from'))
             .predict_model('model', feed_dict=feed_dict, fetches='output_accuracy', save_to=V('accuracy'), mode='a')
            )

Extracting C:\Users\kozhevin\AppData\Local\Temp\train-images-idx3-ubyte.gz
ExtractingExtracting  ExtractingC:\Users\kozhevin\AppData\Local\Temp\train-labels-idx1-ubyte.gzC:\Users\kozhevin\AppData\Local\Temp\t10k-images-idx3-ubyte.gz 

C:\Users\kozhevin\AppData\Local\Temp\t10k-labels-idx1-ubyte.gz


In [16]:
mr = Research()
mr.add_pipeline(train_ppl, 'loss', config=ppl_cfg, preproc=train_preproc, name='train')
mr.add_pipeline(test_ppl, 'accuracy', config=ppl_cfg, preproc=test_preproc, name='test', import_model_from='train')
mr.add_grid_config(grid_config)

In [17]:
start = time()
mr.run(n_reps=2, n_iters=10, n_jobs=1)
print(time() - start)

85.8184916973114


In [13]:
[item.alias() for item in grid_config.gen_configs()]

[{'B': 'cna', 'model': dataset.models.tf.vgg.VGG7},
 {'B': 'can', 'model': dataset.models.tf.vgg.VGG7},
 {'B': 'acn', 'model': dataset.models.tf.vgg.VGG7}]